In [1]:
# --- Imports ---
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


In [2]:

# --- Load & prep ---
df = pd.read_excel(r"C:\Users\SUSHIL KUMAR\Desktop\retailsales_analysis\data\Online Retail.xlsx")
df = df.dropna(subset=['CustomerID','Description'])
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]



In [3]:
# Basic cleaning of product names
df['Description'] = df['Description'].str.strip().str.upper()



In [4]:
# Create basket: InvoiceNo × Description (binary matrix)
# Optional: do per-country to reduce sparsity (use UK as it’s the majority)
country = "United Kingdom"
basket = (df[df['Country'] == country]
          .groupby(['InvoiceNo','Description'])['Quantity']
          .sum().unstack().fillna(0))



In [5]:
# Convert to 0/1
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\SUSHIL KUMAR\AppData\Local\Temp\ipykernel_16536\3503356935.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


In [9]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Filter to UK (consistent customer pattern)
df_uk = df[df['Country'] == "United Kingdom"]

# Remove rarely purchased products (keeps meaningful rules)
product_counts = df_uk['Description'].value_counts()
df_uk = df_uk[df_uk['Description'].isin(product_counts[product_counts > 50].index)]

# Create basket matrix (InvoiceNo x Product)
basket = (df_uk
          .groupby(['InvoiceNo','Description'])['Quantity']
          .sum()
          .unstack()
          .fillna(0))

# Convert to 0/1
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# 🚀 Use FP-Growth instead of Apriori
freq_items = fpgrowth(basket, min_support=0.01, use_colnames=True)

# Sort results
freq_items = freq_items.sort_values('support', ascending=False)

# Generate Rules
rules = association_rules(freq_items, metric="lift", min_threshold=1.0)

rules = rules.sort_values(by=['lift','confidence','support'], ascending=False)

print("Top Product Associations:")
rules.head(15)


C:\Users\SUSHIL KUMAR\AppData\Local\Temp\ipykernel_16536\1437132410.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)
c:\Users\SUSHIL KUMAR\Desktop\retailsales_analysis\venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Top Product Associations:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
929,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010917,0.011039,0.010307,0.944134,85.530202,1.0,0.010186,17.702409,0.999216,0.884817,0.943511,0.938918
928,(HERB MARKER ROSEMARY),(HERB MARKER THYME),0.011039,0.010917,0.010307,0.933702,85.530202,1.0,0.010186,14.918674,0.999340,0.884817,0.932970,0.938918
1021,(HERB MARKER PARSLEY),(HERB MARKER THYME),0.010856,0.010917,0.010063,0.926966,84.913220,1.0,0.009944,13.542834,0.999069,0.859375,0.926160,0.924377
1020,(HERB MARKER THYME),(HERB MARKER PARSLEY),0.010917,0.010856,0.010063,0.921788,84.913220,1.0,0.009944,12.646917,0.999130,0.859375,0.920929,0.924377
975,(HERB MARKER PARSLEY),(HERB MARKER ROSEMARY),0.010856,0.011039,0.010124,0.932584,84.483891,1.0,0.010004,14.669594,0.999008,0.860104,0.931832,0.924856
974,(HERB MARKER ROSEMARY),(HERB MARKER PARSLEY),0.011039,0.010856,0.010124,0.917127,84.483891,1.0,0.010004,11.935675,0.999193,0.860104,0.916218,0.924856
1039,(HERB MARKER BASIL),(HERB MARKER ROSEMARY),0.010917,0.011039,0.010002,0.916201,82.999722,1.0,0.009881,11.801606,0.998856,0.836735,0.915266,0.911139
1038,(HERB MARKER ROSEMARY),(HERB MARKER BASIL),0.011039,0.010917,0.010002,0.906077,82.999722,1.0,0.009881,10.530829,0.998979,0.836735,0.905041,0.911139
1036,(HERB MARKER PARSLEY),(HERB MARKER MINT),0.010856,0.011222,0.010002,0.921348,82.105154,1.0,0.009880,12.571612,0.998662,0.828283,0.920456,0.906326
1037,(HERB MARKER MINT),(HERB MARKER PARSLEY),0.011222,0.010856,0.010002,0.891304,82.105154,1.0,0.009880,9.100128,0.999031,0.828283,0.890111,0.906326


In [10]:
# Save
rules.to_csv(r"C:\Users\SUSHIL KUMAR\Desktop\retailsales_analysis\models\association_rules.csv", index=False)
print("Saved rules to ../models/association_rules.csv")

Saved rules to ../models/association_rules.csv
